# **Projeto 1:** Simulação de Evacuação de Emergência

**Descrição:** Neste projeto, os alunos desenvolverão um modelo para simular a evacuação de pessoas em um edifício durante uma emergência (incêndio, terremoto, etc). Os agentes representarão individuos com diferentes capacidades físicas, níveis de pânico e conhecimento do ambiente

**Objetivos:**
*   Avaliar a eficiência de diferentes estratégias de evacuação;
*   Investigar como a densidade populacional afeta o tempo de evacuação;
*   Analisar o impacto de diferentesa saídas de emergência e obstáculos internos no fluxo de evacuação.

## Instalação de bibliotecas

In [1]:
!pip install agentpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 557.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.8/778.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00


In [2]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import seaborn as sns
import IPython
import random

## Definição dos agentes

In [7]:
ADULT_KEY = 'adult'
EMPLOYEE_KEY = 'employee'
CHILD_KEY = 'child'
ELDER_KEY = 'elder'
LIM_MOB_KEY = 'limited_mobility'
ENV_KNOW_KEY = 'environment_knowledge'
PHYS_CAP_KEY = 'physical_capacity'

AGENTS_CLASS_CHARACTERISTICS_MAPPING = {
    ADULT_KEY: {
        PHYS_CAP_KEY: 3,
        ENV_KNOW_KEY: 2
    },
    EMPLOYEE_KEY: {
        PHYS_CAP_KEY: 3,
        ENV_KNOW_KEY: 3
    },
    CHILD_KEY: {
        PHYS_CAP_KEY: 2,
        ENV_KNOW_KEY: 1
    },
    ELDER_KEY: {
        PHYS_CAP_KEY: 2,
        ENV_KNOW_KEY: 1
    },
    LIM_MOB_KEY: {
        PHYS_CAP_KEY: 1,
        ENV_KNOW_KEY: 1
    }
}

class ObstacleAgent(ap.Agent):
  def setup(self):
    pass

  def agent_method(self):
    # This agent has no action, it only stays in its position
    pass


class EmergencyExitSignAgent(ap.Agent):
  def setup(self):
    # TODO: Ele vai buscar pelas saídas mais próximas e definir a saída mais próxima
    # que será a saída que ele indicará para os agentes
    self.nearest_emergency_exit = (10, 0) # Hardcoded por enquanto

  def setup_pos(self, grid):
    self.grid = grid

  def inform_nearest_emergency_exit(self):
    # Olha os agentes próximos e envia uma mensagem contendo a informação da saída mais próxima

    # Utiliza o protocolo FIPA para indicar a saída
    # TODO: Modificar para utilizar o FIPA
    neighbors = self.grid.neighbors(self, 2)
    for agents in neighbors:
      if agents.agent_class == ADULT_KEY:
        agents.nearest_emergency_exit = self.nearest_emergency_exit


class PersonAgent(ap.Agent):
  # def setup(self, characteristics):
  def setup(self):
    # Preciso definir as características dos agentes de forma aleatoria
    # ou com base em algum parametro
    # self.physical_capacity = self.p.physical_capacity
    # self.panic_level = self.p.panic_level
    # self.environment_knowledge = self.p.environment_knowledge
    # TODO: Esses argumentos serão recebidos como entrada
    # self.physical_capacity = characteristics[PHYS_CAP_KEY]
    # self.panic_level = 0
    # self.environment_knowledge = characteristics[ENV_KNOW_KEY]
    self.physical_capacity = 1
    self.panic_level = 0
    self.environment_knowledge = 1
    self.agent_class = ADULT_KEY
    self.nearest_emergency_exit = None
    self.is_safe = False

  def setup_pos(self, grid):
    self.grid = grid

  def _compute_better_path(self, destination_path):
      if abs(destination_path[0]) >= self.physical_capacity:
        if destination_path[0] > 0:
          x_movement = self.physical_capacity
        else:
          x_movement = -self.physical_capacity
      else:
        x_movement = destination_path[0]

      if abs(destination_path[1]) >= self.physical_capacity:
        if destination_path[1] > 0:
          y_movement = self.physical_capacity
        else:
          y_movement = -self.physical_capacity
      else:
        y_movement = destination_path[1]

      return (x_movement, y_movement)

  def evacuate(self):
    # neighbors = self.building.neighbors(person, 1) # Isso aqui me devolve uma lista de agentes proximos no raio, essa lista pode ser vazia
    # Isso vai ajudar pra ver se preciso me comunicar com algum agente proximo ou se existe algum obstaculo
    # print(f"Agent: {person}, Agent neighbors = {neighbors}, Len = {len(neighbors)}")
    if self.nearest_emergency_exit == None:
      random_x_movement = random.choice([-self.physical_capacity, self.physical_capacity])
      random_y_movement = random.choice([-self.physical_capacity, self.physical_capacity])
      current_relative_destination = (random_x_movement, random_y_movement)
    else:
      current_agent_position = self.grid.positions[self]
      delta_x = self.nearest_emergency_exit[0] - current_agent_position[0]
      delta_y = self.nearest_emergency_exit[1] - current_agent_position[1]
      destination_path = (delta_x, delta_y)
      current_relative_destination = self._compute_better_path(destination_path)

    self.grid.move_by(self, current_relative_destination)

    # TODO: Checar se está perto da saída de emergência
    neighbors = self.grid.neighbors(self, self.physical_capacity)
    for agent in neighbors:
      try:
        if agent.is_emergency_exit:
          self.nearest_emergency_exit = self.grid.positions[agent]
      except:
        pass


class EmergencyExitAgent(ap.Agent):
  def setup(self):
    self.people_passed = 0
    self.is_emergency_exit = True

  def setup_pos(self, grid):
    self.grid = grid

  def allow_people(self):
    neighbors = self.grid.neighbors(self, 1)
    for agent in neighbors:
      if agent.is_safe == False:
        self.people_passed = self.people_passed + 1
        agent.is_safe = True

## Definição do ambiente

In [8]:
## E.g,: The total number of points is equal to width * height
## Let's say width = 10 and height = 10, there will be 100 available points
## If ObjectDensity is equal to 0.1, there will be 10 points with objects
## This will leave us with 100 - 0.1*100 = 90 available points
## Let's say PopulationDensity is equal to 0.5, there will be 0.5*90 = 45 agents
## spread in the available 90 points

class BuildingEvacuationModel(ap.Model):
  def setup(self):
    # Called at the start of the simulation

    # Global values
    width = self.p.width
    height = self.p.height
    number_of_grids = width * height

    # Create grid (building)
    # The building will be represented as a one floor thas specified width and height
    self.building = ap.Grid(self, [width, height], track_empty=True)

    # Place the fire alarms
    # TODO: Pensar numa logica de posicionar as placas de saida de emergencia com base numa lógica
    n_emergency_exit_signs = self.p.n_emergency_exit_signs
    self.emergency_exit_sign = ap.AgentList(self, n_emergency_exit_signs, EmergencyExitSignAgent)
    self.emergency_exit_sign.setup_pos(self.building)
    self.building.add_agents(self.emergency_exit_sign, positions=[(0, int(height/2)), (width - 1, int(height/2))])

    # Place the emergency exits
    # TODO: Pensar numa logica de posicionar os alarmes de incendio
    n_of_emergency_exits = self.p.n_emergency_exit
    self.emergency_exit = ap.AgentList(self, n_of_emergency_exits, EmergencyExitAgent)
    self.emergency_exit.setup_pos(self.building)
    self.building.add_agents(self.emergency_exit, positions=[(int(width/2), 0)])

    # Place the obstacles
    # Comentado por enquanto para ser o cenário inicial
    number_of_obstacles = 0
    # number_of_obstacles = int(self.p.obstacles_density * (number_of_grids - (n_emergency_exit_signs + n_of_emergency_exits)))
    # print(f"Number of obstacles = {number_of_obstacles}")
    # self.objects = ap.AgentList(self, number_of_obstacles, ObstacleAgent)
    # self.building.add_agents(self.objects, random=True, empty=True)

    # Place the agents
    # TODO: Receber o numero de agentes como entrada da simulação
    # TODO: Depois evoluir para definir a quantidade de pessoas de cada classe
    number_of_person_agents = self.p.n_agents
    print(f"Number of agents = {number_of_person_agents}")
    self.person_agents = ap.AgentList(self, number_of_person_agents, PersonAgent)
    self.person_agents.setup_pos(self.building)
    self.building.add_agents(self.person_agents, random=True, empty=True)


  def step(self):
    # Called at every simulation step
    self.emergency_exit_sign.inform_nearest_emergency_exit()
    self.person_agents.evacuate()
    self.emergency_exit.allow_people()

    # Check if the stop criteria was met
    # The stop criteria would be all people saved


  def update(self):
    # Called after setup as well as after each step
    # Normally used to record variables
    pass


  def end(self):
    # Called at the end of the simulation
    pass

## Executando a simulação

In [9]:
# TODO: Rever quais serão os argumentos de entrada
width = 20
height = 5

# Nem todos os argumentos estão sendo utilizados ainda
parameters = {
    'n_agents': 1,
    'adults_percentage': 0.1,
    'employee_percentage': 0.2,
    'child_percentage': 0.3,
    'elder_percentage': 0.2,
    'limited_mobility_percentage': 0.2,
    'n_emergency_exit': 1,
    'n_emergency_exit_signs': 2,
    'steps': 20,
    'width': width,
    'height': height,
}

# parameters = {
#     'width': width,
#     'height': height,
#     'obstacles_density': 0.05,
#     'agents_density': 0.1,
#     'steps': 20
# }

model = BuildingEvacuationModel(parameters)
results = model.run(seed=42)

Number of agents = 1
Completed: 20 steps
Run time: 0:00:00.008467
Simulation finished


In [11]:
def animation_plot(model, ax):
  # The dictionary of agents
  agents_positions = {str(key): value for key, value in model.building.positions.items()}

  # Define colors for each unique type of agent
  AGENT_COLOR_MAPPING = {
      "EmergencyExitSignAgent": "orange",
      "EmergencyExitAgent": "green",
      "ObstacleAgent": "gray",
      "PersonAgent": "blue",
  }

  # Plot each agent
  for agent, (x, y) in agents_positions.items():
      agent_type = agent.split()[0]  # Get the type of agent
      color = AGENT_COLOR_MAPPING.get(agent_type, "grey")  # Get the color for this type of agent
      ax.add_patch(patches.Rectangle((x, y), 1, 1, edgecolor='black', facecolor=color))

  # Set the limits of the plot
  ax.set_xlim(0, width)
  ax.set_ylim(0, height)

  # Add grid
  ax.grid(True)

  # Add legend
  handles = [patches.Patch(color=color, label=agent_type) for agent_type, color in AGENT_COLOR_MAPPING.items()]
  ax.legend(handles=handles, bbox_to_anchor=(0.5, -0.25), loc='upper center')
  ax.set_title(f"Simulation of Evacuation \n"
               f"Time-step: {model.t}")
  ax.set_aspect('equal', adjustable='box')

fig, ax = plt.subplots()
model = BuildingEvacuationModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=5))

Number of agents = 1
